In [3]:
import andi
import numpy as np
import csv as csv
import numpy as np

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
from data_split import data_split

In [ ]:
#from keras.models import Sequential, load_model
from keras.layers import Dense, Conv1D, Dropout, BatchNormalization, Flatten

from keras.regularizers import l2 as regularizer_l2

from keras.optimizers import Adam
#from keras.utlils.np_utils import to_categorical

In [ ]:
os.getcwd()

In [ ]:
AD = andi.andi_datasets()


Importing the data

In [4]:
import csv
trajs_from_files = csv.reader(open('validation_for_scoring/task3.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

Importing the networks

In [5]:
# model_switch_a_t = load_model('task3_a_t.h5')

# model_switch_a_t_rev = load_model('task3_a_t_rev.h5')

#model_switch_a_t_ = tf.keras.models.load_model('task3_int.h5')

model_switch_a_t_new = load_model('task3_new.h5')

model_switch_a_t_diff = load_model('diff_task3.h5')




model_classi_first = load_model('task3_classi_first10249.h5')

model_classi_sec = load_model('task3_classi_second.h5')

Personalized arctan to convert predcitions into switching times stats

In [6]:
def my_atan(x1,x2):
    y=np.arctan2(x1,x2)
    b=y<0
    c=b.astype(int)*(2*np.pi)
    d=y+c 
    return    d;

In [7]:
#for 1d
i=200
validation=validation[0]
test_tim_step=np.arange(200)
show_time_coll=np.tile(test_tim_step,(len(validation),1))
data_show,label_show,traj_show,times_show=data_split(np.asarray(validation),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation)),
                                                         start_row=0,num_row=len(validation),
                                                         traj_len=np.asarray(validation).shape[1],n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)
                                                     
# pred_a_t=model_switch_a_t.predict(data_show)
# pred_a_t_int=model_switch_a_t_int(data_show)
pred_a_t_new=model_switch_a_t_new(data_show)


#model classification

pred_m1_first=np.argmax(model_classi_first.predict(data_show),axis=1)
pred_m2_sec=np.argmax(model_classi_sec.predict(data_show),axis=1)

In [ ]:
validation.shape

In [ ]:
np.diff(np.asarray(validation)[:2,:10],axis=1)

In [ ]:
np.asarray(validation)[:2,:10]

In [8]:
#Estimating the exponents and switching point using diff
i=200

data_show_diff,label_show,traj_show,times_show=data_split(np.diff(np.asarray(validation),axis=1),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation)),
                                                         start_row=0,num_row=len(validation),
                                                         traj_len=i-1,n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)
pred_a_t_diff=model_switch_a_t_diff.predict(data_show_diff)

In [ ]:
#Estimating the exponents and switching point using the reversed trajectory
i=200

data_show_rev,label_show,traj_show,times_show=data_split(np.fliplr(np.asarray(validation)),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation)),
                                                         start_row=0,num_row=len(validation),
                                                         traj_len=i,n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)
pred_a_t_rev=model_switch_a_t_rev.predict(data_show_rev)

In [ ]:
np.mean(np.abs(pred_a_t_int[:,0]-pred_a_t_new[:,0]))

In [ ]:
np.mean(np.abs(pred_a_t_int[:,0]-pred_a_t[:,0]))

In [9]:
#combining predictions for the switching time
# pr_t = my_atan(pred_a_t[:,2],pred_a_t[:,3])*200/(2*np.pi)

# pr_t_rev = my_atan(pred_a_t_rev[:,2],pred_a_t_rev[:,3])*200/(2*np.pi)

# pr_t_int = my_atan(pred_a_t_int[:,2],pred_a_t_int[:,3])*200/(2*np.pi)

pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)

pr_t_diff = my_atan(pred_a_t_diff[:,2],pred_a_t_diff[:,3])*200/(2*np.pi)




pr_t_comb = (pr_t_new+pr_t_diff)/2

In [10]:
pred_m1s=pred_m1_first
pred_m2s=pred_m2_sec

In [11]:
#assembling the total prediction array now giving (t,model 1,a1,model2,a2)
predictions=np.zeros((len(validation),5))
predictions[:,0]=pr_t_comb
predictions[:,1]=pred_m1s
predictions[:,2]=(pred_a_t_new[:,0]+pred_a_t_diff[:,0])/2
predictions[:,3]=pred_m2s
predictions[:,4]=(pred_a_t_new[:,1]+pred_a_t_diff[:,1])/2


In [12]:
pred_to_txt = np.ones((len(predictions), 6))
for i in range(len(predictions)):
    for j in range(5):
        
        pred_to_txt[i, j+1] = predictions[i][j]

np.savetxt('task3.txt', pred_to_txt.astype(float), fmt = '%1.5f', delimiter = ';')